In [1]:
import pandas as pd

In [2]:
raw_data = pd.read_csv("conversion_data.csv")

In [3]:
raw_data.head(2)

,country,age,new_user,source,total_pages_visited,converted
0,UK,25,1,Ads,1,0
1,US,23,1,Seo,5,0


In [ ]:
#check whether the dataset is balanced or not. 